<a href="https://colab.research.google.com/github/ashraf-ul/DeepLearningWithTensorflow/blob/master/Copy_of_GloVe_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis with GloVe Vectors

Sentiment analysis is the process of categorizing opinions in natural language text. Several methods can be used to estimate sentiment. In this example, labeled reviews from Yelp, Amazon, and IMDB are used to train a supervised binary classification model. The model is a 1D convolutional neural network (CNN). While 2D CNNs are commonly used for image classification, their exceptional spatial capabilities can applied to text in one dimension.

Words in a sentence must be encoded as vectors for training and prediction. This encoding is more commonly called embedding. A straightforward approach would assign every distinct word a unique numerical value. A better approach is use pretrained word embeddings based on a large corpus of text, such as Wikipedia. Global Vectors for Word Representation ([GloVe](https://nlp.stanford.edu/projects/glove/)) is a popular vector representation based on word co-occurance.


In [0]:
%tensorflow_version 2.x

In [0]:
import pandas as pd
import numpy as np
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# Training Data

Data is loaded into a Pandas data frame from text files. This dataset contains two columns: a natural language comment and binary positive/negative sentiment represented as 1 or 0.

The data is available from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences)

In [0]:
# Download data from https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences

!wget --quiet "https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip"
!unzip -q "sentiment labelled sentences"
!mv "sentiment labelled sentences" data
!ls -l data

In [0]:
!cat data/readme.txt

In [0]:
files = ['data/yelp_labelled.txt', 'data/amazon_cells_labelled.txt', 'data/imdb_labelled.txt']

df_list = []
for file in files:
    df = pd.read_csv(file, names=['comment', 'sentiment'], sep='\t')
    df_list.append(df)

trainingData = pd.concat(df_list)

print('Number of rows: %d' % len(trainingData))

trainingData.head(10)

# GloVE Embeddings

These are some convenience functions for loading GloVE vectors and creating an embedding matrix.

The GloVe vector files are downloaded from [Stanford](https://nlp.stanford.edu/projects/glove/).

In [0]:
# Download GloVe embeddings

!wget http://nlp.stanford.edu/data/glove.6B.zip
!echo "Unzipping glove.6B.zip"
!unzip -q glove.6B.zip
!echo "All done!"

In [0]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 50

num_classes = 2

def load_glove_vectors():
    print('Loading glove vectors...')
    glove_map = {}
    with open('glove.6B.%dd.txt' % EMBEDDING_DIM, encoding='utf8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            glove_map[word] = np.asarray(values[1:], dtype='float32')
    return glove_map

def create_embedding_matrix(word_index, num_words):
    glove_map = load_glove_vectors()
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        if i > num_words:
            continue
        vector = glove_map.get(word)
        if vector is not None:
            embedding_matrix[i] = vector
    return embedding_matrix

# Encode Text

In [0]:
comments = trainingData.comment.astype(str).tolist()
sentiments = trainingData.sentiment.tolist()
labels = np.asarray(sentiments)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(comments)
sequences = tokenizer.texts_to_sequences(comments)
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index

num_words = min(MAX_NB_WORDS, len(word_index)) + 1

x_train, x_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2)

In [0]:
embedding_matrix = create_embedding_matrix(word_index, num_words)

# Define Model

This is a 1D CNN model with a Keras embedding layer using the embedding matrix created above. The embedding layer is pre-trained, so it will not be trained here.

In [0]:
dropout = 0.4

model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model.add(Dropout(dropout))

model.add(Conv1D(128, 5, activation='relu', padding='same', strides=2))
model.add(GlobalMaxPooling1D())
model.add(Dropout(dropout))

model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['acc'])

# Train model for a given number of epochs
history = model.fit(x_train, y_train, batch_size=128, epochs=40, verbose=1, validation_data=(x_test, y_test))

# Evaluate model against test data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Prediction

In [0]:
predictions = model.predict(padded_sequences)
most_likely = predictions.argmax(1)

In [0]:
index = random.randrange(len(predictions))
print(comments[index])
print('Prediction: %d, label: %d' % (most_likely[index], sentiments[index]))

# Error Analysis

In [0]:
for i in range(10000):
    index = random.randrange(len(predictions))
    if most_likely[index] != sentiments[index]:
        break

print(comments[index])
print('Prediction: %d, label: %d' % (most_likely[index], sentiments[index]))

plt.bar(range(num_classes), predictions[index], tick_label=range(num_classes))
plt.title('Prediction values')
plt.show()